In [1]:
import pandas as pd
import numpy as np
import re

In [ ]:
# pip install openpyxl

In [42]:
df = pd.read_excel(r'C:\Users\wasia\Desktop\OFAC data listing\raw_ofac_list.xlsx')
df

,list,source
0,"ALIZADEH HOSHYAR, Javad (Arabic: جواد عليزاده ...",https://ofac.treasury.gov/recent-actions/20250731
1,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD...,https://ofac.treasury.gov/recent-actions/20250731
2,"CLIFTON TRADING LIMITED, Unit 04-05, 16th Floo...",https://ofac.treasury.gov/recent-actions/20250731
3,CONTROL AFZAR TABRIZ CO LTD (Arabic: شرکت کنتر...,https://ofac.treasury.gov/recent-actions/20250731
4,JOEMARS MACHINERY AND ELECTRIC INDUSTRIAL CO L...,https://ofac.treasury.gov/recent-actions/20250731
5,MECATRON MACHINERY CO LTD (Chinese Traditional...,https://ofac.treasury.gov/recent-actions/20250731


In [ ]:
# function to extract name and alias from ofac raw file


def extract_name_alias(text):
    if not isinstance(text, str) or not text.strip():
        return pd.Series({'name': '', 'alias': ''})
    
    all_brackets = re.findall(r'\(([^()]*)\)', text)
    if not all_brackets:
        return pd.Series({'name': '', 'alias': ''})
    
    first_bracket = all_brackets[0]
    
    # If first () contains Linked To:, return empty
    if 'Linked To:' in first_bracket:
        return pd.Series({'name': '', 'alias': ''})
    
    # If first () does NOT contain a.k.a. or f.k.a., and does NOT contain colon, return empty
    if ('a.k.a.' not in first_bracket and 'f.k.a.' not in first_bracket) and (':' not in first_bracket):
        return pd.Series({'name': '', 'alias': ''})
    
    name = text.split('(', 1)[0].strip()
    
    alias_list = []
    if 'a.k.a.' not in first_bracket and 'f.k.a.' not in first_bracket:
        alias_list.append(first_bracket.strip())
    
    aka_matches = re.findall(r'\(([^()]*(?:a\.k\.a\.|f\.k\.a\.).*?)\)', text)
    for match in aka_matches:
        parts = re.split(r'a\.k\.a\.|f\.k\.a\.', match)
        for part in parts[1:]:
            aliases = [p.strip() for p in part.split(';') if p.strip()]
            for alias in aliases:
                nested = re.findall(r'\(([^()]*)\)', alias)
                if nested:
                    alias_main = re.sub(r'\([^()]*\)', '', alias).strip()
                    alias_list.append(alias_main)
                    for n in nested:
                        alias_list.append(n.strip())
                else:
                    alias_list.append(alias)
    
    # Deduplicate aliases preserving order
    seen = set()
    final_aliases = []
    for a in alias_list:
        if a not in seen:
            seen.add(a)
            final_aliases.append(a)
    
    return pd.Series({'name': name, 'alias': '; '.join(final_aliases)})

# Usage example:
# Suppose your DataFrame is df and the column with text is 'text_column'

df[['name', 'alias']] = df['list'].apply(extract_name_alias)

# Now df has two new columns 'name' and 'alias' with extracted info


In [39]:
df

,list,link,name,alias
0,"ALIZADEH HOSHYAR, Javad (Arabic: جواد عليزاده ...",https://ofac.treasury.gov/recent-actions/20250731,"ALIZADEH HOSHYAR, Javad","Arabic: جواد عليزاده هوشيار; ALIZADE HOUSHYAR,..."
1,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD...,https://ofac.treasury.gov/recent-actions/20250731,CHANGZHOU JOEMARS INDUSTRIAL AUTOMATION CO LTD,Chinese Simplified: 常州乔懋工业自动化有限公司; CHANGZHOU J...
2,"CLIFTON TRADING LIMITED, Unit 04-05, 16th Floo...",https://ofac.treasury.gov/recent-actions/20250731,,
3,CONTROL AFZAR TABRIZ CO LTD (Arabic: شرکت کنتر...,https://ofac.treasury.gov/recent-actions/20250731,CONTROL AFZAR TABRIZ CO LTD,Arabic: شرکت کنترل افزار تبريز
4,JOEMARS MACHINERY AND ELECTRIC INDUSTRIAL CO L...,https://ofac.treasury.gov/recent-actions/20250731,,
5,MECATRON MACHINERY CO LTD (Chinese Traditional...,https://ofac.treasury.gov/recent-actions/20250731,MECATRON MACHINERY CO LTD,Chinese Traditional: 龍馬精機股份有限公司


In [41]:

df.to_csv('df_check.csv', encoding='utf-8-sig', index=False)